In [13]:
import tensorflow as tf

from pathlib import Path
from keras import layers, Sequential, backend
from keras.utils import layer_utils
from keras.models import load_model

In [2]:
def VGG16():
    input_shape = (224, 224, 3)
    
    model = Sequential(
        [
            layers.Input(shape=input_shape),
            
            layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.MaxPool2D(pool_size=(2,2), strides=(2,2), trainable=False),

            layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.MaxPool2D(pool_size=(2,2), strides=(2,2), trainable=False),

            layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.MaxPool2D(pool_size=(2,2), strides=(2,2), trainable=False),

            layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.MaxPool2D(pool_size=(2,2), strides=(2,2), trainable=False),

            layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", trainable=False),
            layers.MaxPool2D(pool_size=(2,2), strides=(2,2), name='vgg16', trainable=False),
        ]
    )

    weights_path = str(Path.cwd()) + r"\Model_Data\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
    model.load_weights(weights_path)

    if backend.backend() == 'theano':
        layer_utils.convert_all_kernels_in_model(model)

    data_augmentation = Sequential(
        [
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
        ]
    )
    
    preprocess_input = tf.keras.applications.vgg16.preprocess_input
    global_average_layer = layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(15, activation='softmax')
    
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = model(x)
    x = global_average_layer(x)
    x = layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
        
    model = tf.keras.Model(inputs, outputs)

    return model

In [3]:
model = VGG16()

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
              
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 sequential (Sequential)     (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                      

In [5]:
train_dir = str(Path.cwd()) + r"\Dataset\train"
validation_dir = str(Path.cwd()) + r"\Dataset\test"

model_save = str(Path.cwd()) + r"\Model_Data\Model.h5"

In [6]:
batch = 64
img_size = (224, 224)

autotune = tf.data.AUTOTUNE

In [7]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, labels='inferred', label_mode='int', shuffle=True, batch_size=batch, image_size=img_size)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir, labels='inferred', label_mode='int', shuffle=True, batch_size=batch, image_size=img_size)

Found 27494 files belonging to 15 classes.
Found 3725 files belonging to 15 classes.


In [8]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

train_dataset = train_dataset.prefetch(buffer_size=autotune)
validation_dataset = validation_dataset.prefetch(buffer_size=autotune)
test_dataset = test_dataset.prefetch(buffer_size=autotune)

In [9]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save, verbose=1, monitor='val_accuracy', mode='max', save_best_only=True)
es_callback = tf.keras.callbacks.EarlyStopping(patience=3, verbose=1)

In [11]:
model.fit(train_dataset, epochs=1000, validation_data=validation_dataset, callbacks=[cp_callback, es_callback])

Epoch 1/1000
430/430 [==============================] - ETA: 0s - loss: 5.8481 - accuracy: 0.2085
Epoch 1: val_accuracy improved from -inf to 0.45680, saving model to D:\Projects\Human_Animal_Face\Model\Model_Data\Model.h5
430/430 [==============================] - 291s 648ms/step - loss: 5.8481 - accuracy: 0.2085 - val_loss: 2.8328 - val_accuracy: 0.4568
Epoch 2/1000
430/430 [==============================] - ETA: 0s - loss: 2.7617 - accuracy: 0.4578
Epoch 2: val_accuracy improved from 0.45680 to 0.69613, saving model to D:\Projects\Human_Animal_Face\Model\Model_Data\Model.h5
430/430 [==============================] - 270s 626ms/step - loss: 2.7617 - accuracy: 0.4578 - val_loss: 1.3046 - val_accuracy: 0.6961
Epoch 3/1000
430/430 [==============================] - ETA: 0s - loss: 1.8878 - accuracy: 0.5895
Epoch 3: val_accuracy improved from 0.69613 to 0.79113, saving model to D:\Projects\Human_Animal_Face\Model\Model_Data\Model.h5
430/430 [==============================] - 269s 625ms/s

In [14]:
model = load_model(model_save)

In [15]:
model.evaluate(validation_dataset)

48/48 [==============================] - 27s 532ms/step - loss: 0.1980 - accuracy: 0.9407


[0.19803258776664734, 0.9407480955123901]